In [21]:
import requests
import pandas as pd
def query_openai(prompt):
    api_key = os.getenv("OPENAI_API_KEY")
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json',
    }

    # Proper structure for chat-based models
    data = {
        'model': 'gpt-3.5-turbo',
        'messages': [
            {"role": "user", "content": prompt}
        ],
        'max_tokens': 4096
    }

    response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, json=data)
    response_data = response.json()
    if 'error' in response_data:
        print("Error from API:", response_data['error'])
        return None
    return response_data

def get_response(query,question,answer):
    instructions = """
       Analyze each question and answer, if the question does not make sense change it. change every answer to precise factual answer.
       
    """
    
    q = f""" these are the instructions: {instructions}.
        this is the question: {question}.
        answer I have: {answer}.
        for questions like these : Who won the NBA championship in Heat? THEY DO NOT MAKE SENSE CONVERT THEM TO WHEN AND ALSO.
        INCLUDE QUESTIONS WITH PERNCETAGES AS ANSWERS.
        LET YOUR OUTPUT BE IN THIS FORMAT NOTHING MORE 
            CORRECTED QUESTION:
            ANSWER:
            DO NOT INCLUDE MORE THAN THESE 2
    """
    response = query_openai(q)
    response_message = response['choices'][0]['message']['content']
    return response_message

In [3]:
df=pd.read_csv("Final_Refined_Unique_Basketball_QnA_for_LLaMA_Finetuning.csv")

In [22]:
responses=[]
for question,answer in zip(df['Question'],df['Answer']):
    response=get_response(None,question,answer)
    responses.append(response)


In [ ]:
newdf=pd.DataFrame()
ques=[]
anss=[]
i=0
for text in responses:
    qu=text.split("\n")
    i+=1
    if len(qu)==2:
        QUESTION=qu[0]
        ANSWER=qu[1].replace("Answer: ","")
        ques.append(QUESTION)
        anss.append(ANSWER)
    else:
        print(text+"\n\n")
        print(len(qu),qu)
newdf["Question"]=ques
newdf["Answer"]=anss